In [1]:
import os,sys
from sklearn.utils import shuffle
f = open("wang_etal_motifs.meme")
f.readline()

motifs = []
for r in f:
    if r.startswith("MOTIF"):
        motifs.append((r.strip(), int(r.split()[4])))

f.seek(0)
probs = []
is_prop = False
added_width = 0
curr_width = -1
for r in f:
    if r.startswith("MOTIF"):
        curr_width = int(r.strip().split()[4])
    elif r.startswith("letter-probability"):
        is_prop = True
        continue
    if is_prop and added_width < curr_width and r.strip() != "" and len(r.strip().split()) == 4:
        probs.append(" ".join(shuffle(r.strip().split(), random_state=42)))
        added_width += 1
    if added_width == curr_width and is_prop:
        is_prop = False
        added_width = 0
        curr_width = -1

letter_prob_lines = []
f.seek(0)
for r in f:
    if r.startswith("letter-proba"):
        letter_prob_lines.append(r.strip())

meta_lines = []
f.seek(0)
for r in f:
    if not r.startswith("MOTIF"):
        meta_lines.append(r.strip())
    else:
        break


probs_shuffled = shuffle(probs, random_state = 42)


In [2]:
import numpy as np
probs_array = []
for p in probs_shuffled:
    probs_array.append([float(i) for i in p.strip().split()])
probs_array = np.array(probs_array)
probs_array.sum(axis=0)


array([ 307.049179,  248.156765,  264.806244,  317.987824])

In [3]:

out = open("wang_motifs_all_shuffled.meme", "w")
for l in meta_lines:
    out.write(l.strip() + "\n")

last_i = 0
for m, l in zip(motifs, letter_prob_lines):
    out.write(m[0] + "\n\n")
    out.write(l + "\n")
    out.write("\n".join([" ".join(map(lambda x: "%.6f"%(x), x)) for x in probs_array[last_i:(last_i + m[1])]]))
    out.write("\n\n\n")
    last_i += m[1]
out.close()